In [2]:
# Установка зависимостей (для RAG, веб-скрапинга и Vosk)
!pip install openai pymupdf numpy requests beautifulsoup4 vosk pydub docling
# Скачиваем небольшую русскую модель Vosk
!wget -nc https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
# Распаковываем и переименовываем папку для удобства
!unzip -n vosk-model-small-ru-0.22.zip
!mv vosk-model-small-ru-0.22 vosk-model-folder

# Импорт необходимых модулей
import fitz  # Главный модуль PyMuPDF
import os # Стандартный модуль для взаимодействия с операционной системой
import numpy as np # Библиотека для научных вычислений
import requests # Библиотека для выполнения HTTP-запросов
import json # Модуль для работы с данными в формате JSON
import wave # Стандартный модуль для чтения и записи файлов в формате WAV
from bs4 import BeautifulSoup # Основной класс библиотеки BeautifulSoup 4
from io import BytesIO # Класс из модуля io для работы с бинарными данными в памяти как с файлом
from openai import OpenAI # Клиентская библиотека для взаимодействия с OpenAI API
from vosk import Model, KaldiRecognizer # Класс для загрузки языковой модели Vosk и класс, который выполняет непосредственно распознавание речи
from google.colab import files # Для загрузки файлов
from pydub import AudioSegment # Класс для манипуляций с аудио (обрезка, конвертация, изменение параметров)

print("✅ Зависимости установлены.")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.8/274.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


**PDF**

In [3]:
# Блок загрузки для PDF
print("Начало загрузки файла...")
uploaded = files.upload()

# Сохраняем имя файла для дальнейшей работы

#Берем первый файл
pdf_for_docling = list(uploaded.keys())[0]

print(f"✅ Файл '{pdf_for_docling}' успешно загружен.")

Начало загрузки файла...


Saving Stax_Tower_B_-_601_pdf.pdf to Stax_Tower_B_-_601_pdf.pdf
✅ Файл 'Stax_Tower_B_-_601_pdf.pdf' успешно загружен.


In [4]:

from docling.document_converter import DocumentConverter
# Инициализируем конвертер
converter = DocumentConverter()

# Запускаем конвертацию
result = converter.convert(pdf_for_docling)

# result.document - это и есть тот самый DoclingDocument
# Его можно экспортировать в разные форматы
markdown_output = result.document.export_to_markdown()
markdown_output


[INFO] 2025-12-10 15:56:37,714 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-10 15:56:37,720 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-10 15:56:37,723 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-10 15:56:41,622 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2025-12-10 15:56:41,981 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-10 15:56:41,982 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-10 15:56:42,168 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-10 15:56:42,169 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-10 15:56:42,170 [RapidOCR] download_file.py:68: Initiating download: https://

'## SCHEDULE OF INSTALMENT PAYMENTS\n\nDate: 21/11/2025\n\n## Dear Valued Customer,\n\n## Thank you for your interest in our project. Below is the payment plan schedule for your reference.\n\n| Unit Number          | Tower B - 601   |\n|----------------------|-----------------|\n| Unit Price           | 949,470.00      |\n| Suite (Sq.ft)        | 495.03          |\n| Balcony Area (Sq.ft) | 266.73          |\n| Total Area (Sq.ft)   | 761.76          |\n\n* Applicable fees to Dubai Land Department are: 4% of the property net price + AED 2,750 Oqood fee.\n\n* Prices and availability are subject to change without notice.\n\n* The estimated completion date is a mere estimate and subject to change from time to time\n\n## Payment Plan\n\n| Instalment                       | Date       | (%)   | Amount (AED)   |\n|----------------------------------|------------|-------|----------------|\n| Down payment                     | 21/11/2025 | 10%   | 94,947.00      |\n| 1st Installment              

In [5]:
print(markdown_output)

## SCHEDULE OF INSTALMENT PAYMENTS

Date: 21/11/2025

## Dear Valued Customer,

## Thank you for your interest in our project. Below is the payment plan schedule for your reference.

| Unit Number          | Tower B - 601   |
|----------------------|-----------------|
| Unit Price           | 949,470.00      |
| Suite (Sq.ft)        | 495.03          |
| Balcony Area (Sq.ft) | 266.73          |
| Total Area (Sq.ft)   | 761.76          |

* Applicable fees to Dubai Land Department are: 4% of the property net price + AED 2,750 Oqood fee.

* Prices and availability are subject to change without notice.

* The estimated completion date is a mere estimate and subject to change from time to time

## Payment Plan

| Instalment                       | Date       | (%)   | Amount (AED)   |
|----------------------------------|------------|-------|----------------|
| Down payment                     | 21/11/2025 | 10%   | 94,947.00      |
| 1st Installment                  | 21/12/2025 | 10%   | 

**WEB PAGE**

In [30]:
url = "https://vk.com/@health_and_intellect-lokalnyi-parsing-dokumentov-dlya-ii-docling"
converter = DocumentConverter()
result_for_website = converter.convert(url)
print(result_for_website.document.export_to_markdown())

# Локальный парсинг документов для ИИ || Docling

[ЗДОРОВЬЕ и ИНТЕЛЛЕКТ](/health_and_intellect) · 22 Oct at 12:56 pm

Локальный парсинг документов для ИИ || Docling, image #1

<!-- image -->

Проблема в том, что большинство документов - PDF, Word, HTML - не имеют четкой структуры, понятной нейросетям. Коммерческие парсеры решают эту задачу, но возникают вопросы безопасности данных, стоимости и качества обработки сложных элементов вроде таблиц и графиков. Здесь на помощь приходит Docling - открытый инструмент для локального парсинга документов.

### Что такое Docling

Docling - это open-source проект от IBM Research, предназначенный для преобразования документов различных форматов в структурированные данные, готовые для работы с ИИ. Инструмент поддерживает PDF, Word (DOCX), PowerPoint (PPTX), HTML и даже изображения, конвертируя их в Markdown или JSON. Проект набрал огромную популярность на GitHub благодаря продвинутым возможностям и полностью бесплатному локальному использованию.

### 